# Template to extract trajectories from raw data

This Notebook is a template to extract one-dimensional cell migration data from a full experiment.

The raw data has to be in Nikon's nd2-format. The data consits of a pattern file that contains one image of the fluorescently labelled patterned lanes per field of view, and of a time-lapse file. The field of views as well as the order has to be the same for the file containing the pattern and the file containing the time-lapse acqusition. The path to these two files has to be specified by the user.

The default values for running the pipeline are stored as onedcelltrack/onedcelltrack/default_pipeline_arguments.json
The arguments used for the running of this pipeline are saved in extraction/pipeline_arguments.json

In [ ]:
import sys
import os
import errno
import time
from onedcelltrack.main_pipeline import Pipeline
from nd2reader import ND2Reader
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib ipympl
%load_ext autoreload

In [ ]:
%autoreload
pipe = Pipeline(pipeline_arguments='extraction/pipeline_arguments.json')
#pipe=Pipeline(pipeline_arguments=None)

## Path to files - input paths

In [ ]:
#Path to folder in which the experiment is contained
data_path = ''

#Name of main time-lapse experiment file 
nd2_file = ''

#Name of file containing patterns
lanes_file = '' 
    
#Path to folder where results should be output
path_out = './extraction/' # in Auswertungsordner

#update the pipeline with files and check everything is fine
pipe.update_file_names(data_path, lanes_file=lanes_file, path_out=path_out, image_file=nd2_file);

## First set up the general pipeline arguments
* Chose the right brightfield and nucleus channels.
* Set the initial and last frame to evaluate.
* Delete any bad fields of view.
* Set the time resolution.
* Don't forget to save the changes, if all goes well, the "Save" button should turn green.

If the next cell doesn't show an image, please run it again: you should then see an image of your experiment

In [ ]:
pipe.Viewer()

## Lane detection - check and optimise

- v=0.3 sweet spot
- clip usually between 0-1800

Note that you must run recompute before the values are updated and can be saved

In [ ]:
pipe.LaneViewer()

## Trackpy - check and optimise
Make Sure that you have chosen the right nucleus channel in advance, or else you might run into problems! 

In [ ]:
pipe.TrackingViewer()

## Cellpose - check and optimise
You can use a self trained model if you like. Otherwise the default is phcmarch23
Optimise the following 3 parameters:
- diameter; between 29-45
- flow_threshold; 1.25 is good starting point. Not really necessary to change
- cellprob_threshold: standard = 0. Not really necessary to change

Remember to save the settings!

In [ ]:
pipe.CellposeViewer(pretrained_model=None)

## Fill up the metadata in an extra csv file (optional)

In [ ]:
#Access metadata from nd2 file
f = ND2Reader(os.path.join(data_path, nd2_file))
metadata = f.metadata

In [ ]:
#Fill in data about experiment
if metadata['date'] is None:
    date = 'None'
else:
    date = metadata['date'].strftime('%d-%m-%Y %H:%m')
    
Experiment_data = {
    #'Experiment_id':,
    'Path':data_path,
    'Date': date,
    'celltype': 'MDA-MB-231',
    'microscope': 'TIRF',
    'nframes': f.sizes['t'],
    'nfov': f.sizes['v'],
    'channels': str(metadata['channels']),
    'fpm': 0.5,
    'pixels/um': f.metadata['pixel_microns'],
    'bitsperpixel': 16,
    'Author': 'Johannes Heyn'}

In [ ]:
Experiment_data = pd.DataFrame(Experiment_data, index =[0])
Experiment_data.to_csv(f'Experiment_data.csv')

## Run the pipeline
All the arguments should be set and optimised by now. In the next step you can run the pipeline which will extract the position of the front, nucleus and back of the cell and save it in a csv file in a folder named after the field of view from where the data originates. More specific progress is logged to the output directory.

In [ ]:
pipe.run_pipeline(run_lane_detection=True, run_segmentation=True, run_tracking=True, convert_to_um=True, classify_trajectories=True, merge_trajectories=True, verbose=True)